challenges:

1- how use vrt for geolocation? can we use it at all?

2- how use vrt for making mosaic?

 <SRS>GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]</SRS>
 
 <GeoTransform> -3.0531428271702840e+01,  3.7890083929483308e-02,  0.0000000000000000e+00,  6.7079735828607269e+01,  0.0000000000000000e+00, -3.7890083929483308e-02</GeoTransform>

Q- SRS? GeoTransform?
Q- can I make vrt and build mosaic at the same time?


steps:
1- build vrt (virtual raster) from list of single raster files using my own code
2- open vrt and add a pixel function to it
3- transform single rasters to mosaic of rasters using vrt using gdal.translate() and vrt

In [1]:
import glob, os
from osgeo import gdal
from subprocess import call
import datetime as dt
import numpy as np

In [2]:
gdal.__version__

'3.4.2'

In [3]:
# we find this file pattern

raster_file_pattern = 'raster_path_*'+'*_reprojToEPSG_3995.tif'

print('looking for pattern: %s' % raster_file_pattern)

looking for pattern: raster_path_**_reprojToEPSG_3995.tif


In [4]:
# path to date dir with raster.tif files there (example with 2 rasters)

raster_dir_fullpath = "/Volumes/Ehsan-7757225325/2016/april_2016/predict_roughness_k_zero_npts_10/roughness_subdir_2016_4_15/test_path_69_85"

print('raster dir: %s' %os.path.isdir(raster_dir_fullpath))



raster dir: True


In [5]:
# day label

day='15'
month='april'
year='2016'



# form date-tag from 3 parameters
date_tag = day+'_'+month+'_'+year

# naming labels
resamplingAlg = 'nearest'
output_VRT_dataset_name = 'virtualDataset_float64_'+resamplingAlg+'_'+date_tag+'_pixelaveraged.vrt'


In [6]:
VRT_fullpath = os.path.join(raster_dir_fullpath, output_VRT_dataset_name)
print(output_VRT_dataset_name)
print('VRT available: %s' %os.path.isfile(VRT_fullpath))

virtualDataset_float64_nearest_15_april_2016_pixelaveraged.vrt
VRT available: True


In [7]:
output_mosaic_name = 'mosaic_fromVRT_float64_'+resamplingAlg+'_'+date_tag+'_pixelaveraged.tif'
print(output_mosaic_name)


mosaic_fromVRT_float64_nearest_15_april_2016_pixelaveraged.tif


In [8]:
# build a list from input raster files

files_to_mosaic = glob.glob(os.path.join(raster_dir_fullpath, raster_file_pattern))
print('raster files found: %d' % len(files_to_mosaic))

raster files found: 73


## after running the following section, turn it off, or move the rest of the code to another notebook

In [9]:
# #~ method 2- VRT and gdal_translate

# print('building VRT dataset!')
# vrt_options = gdal.BuildVRTOptions(resampleAlg='linear') #, addAlpha=True)
# # vrt_options

# my_vrt_ptr = gdal.BuildVRT(VRT_fullpath, 
#                             files_to_mosaic,
#                             srcNodata = -99.0,
#                             VRTNodata = -99.0)#, options=vrt_options)


# # print(my_vrt_ptr)
# print('closing VRT dataset!')
# my_vrt_ptr = None 

# now open vrt and add pixel function

## finally transform single rasters to mosaic of rasters 
we do the transfer using vrt file and gdal.translate() and vrt: translate() does the transformation: when it wants to open each raster, it checks the vrt and reads the raster based on the information inside vrt

In [10]:
# name of mosaic
out_mosaic_fullpath = os.path.join(raster_dir_fullpath, output_mosaic_name)
out_mosaic_fullpath

'/Volumes/Ehsan-7757225325/2016/april_2016/predict_roughness_k_zero_npts_10/roughness_subdir_2016_4_15/test_path_69_85/mosaic_fromVRT_float64_nearest_15_april_2016_pixelaveraged.tif'

In [11]:
print('building mosaic from VRT dataset')
print(VRT_fullpath)

gdal.SetConfigOption('GDAL_VRT_ENABLE_PYTHON', "YES")
# translate_options = gdal.TranslateOptions('GDAL_VRT_ENABLE_PYTHON', "YES")

mosaic_ds = gdal.Translate(out_mosaic_fullpath,
                            VRT_fullpath,
                            format = 'GTiff',
                            noData = -99.0,
                            resampleAlg = resamplingAlg,
                            #options = translate_options,
                            outputType = gdal.GDT_Float64) # note: input dtype is float_64==double, maybe here change dtype to make it smaller img???? # outputType = gdal.GDT_Byte 	
                             


print('\noutput mosaic: ')
print(out_mosaic_fullpath)


building mosaic from VRT dataset
/Volumes/Ehsan-7757225325/2016/april_2016/predict_roughness_k_zero_npts_10/roughness_subdir_2016_4_15/test_path_69_85/virtualDataset_float64_nearest_15_april_2016_pixelaveraged.vrt


ERROR 1: PROJ: proj_create_from_database: /Applications/anaconda3/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.
Warning 1: PROJ: proj_create_from_database: /Applications/anaconda3/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.
Warning 1: PROJ: proj_create_from_database: /Applications/anaconda3/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 0 whereas a number >= 2 is expected. It comes from another PROJ installation.
ERROR 1: <class 'AttributeError'>, module 'gdal_vrt_module_0x7fa98d7ef980' has no attribute 'averageEM'



output mosaic: 
/Volumes/Ehsan-7757225325/2016/april_2016/predict_roughness_k_zero_npts_10/roughness_subdir_2016_4_15/test_path_69_85/mosaic_fromVRT_float64_nearest_15_april_2016_pixelaveraged.tif


## Properly close the datasets to flush to disk

In [12]:
print('closing mosaic dataset!')
mosaic_ds = None
VRT_fullpath = None  # close opened input file

closing mosaic dataset!


In [13]:
# from IPython.display import Audio

# sound_file = 'https://music.apple.com/us/album/beep-version-7-interface-multimedia-software-computer/581961437?i=581961750'

# Audio(sound_file, autoplay=True)

